In [8]:
data_source = ['math-amc', 'math-aime', 'gpqa']

In [20]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")





In [35]:
import json
data = []
with open('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/train/verlCheckpoint/Summary/qwen4b_dapo_math_10k_context_distributed_reward_with_summary/rollout/val_160.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        

In [36]:
import numpy as np

accuracy_by_source = {}
for line in data:
    if line['data_source'] not in accuracy_by_source:
        accuracy_by_source[line['data_source']] = []
    accuracy_by_source[line['data_source']].append(line['final_reward'])
    
for source in accuracy_by_source:
    mean_accuracy = np.mean(accuracy_by_source[source])
    print(source, mean_accuracy, len(accuracy_by_source[source]))
    
    

math-amc 0.7349397590361446 83
gpqa 0.43147208121827413 197
math-aime 0.3707865168539326 89


In [37]:
import numpy as np
response_length_list = {}
for line in data:
    if line['data_source'] not in response_length_list:
        response_length_list[line['data_source']] = []
    response_length_list[line['data_source']].append(len(tokenizer.encode(line['response'])))
    
for source in response_length_list:
    mean_response_length = np.mean(response_length_list[source])
    print(source, mean_response_length)
    
    
    
            
            
            

math-amc 6172.722891566265
gpqa 6799.66497461929
math-aime 8532.0


In [24]:
line['response']

"<think>\nOkay, so I need to find the area of triangle ABC, which is an isosceles right triangle with a right angle at A. There's a point P inside the triangle such that the angles PAB, PBC, and PCA are all equal. Also, AP is given as 10. Hmm, okay. Let me try to visualize this.\n\nFirst, since ABC is an isosceles right triangle with the right angle at A, that means sides AB and AC are equal in length, and BC is the hypotenuse. Let me denote AB = AC = x, so BC would be x√2. The area of ABC would then be (x²)/2, so if I can find x, I can compute the area.\n\nNow, there's a point P inside the triangle such that angle PAB = angle PBC = angle PCA. Let me denote this common angle as θ. So, angle PAB = θ, angle PBC = θ, angle PCA = θ. Also, AP = 10. I need to relate these angles and the length AP to find x.\n\nSince ABC is a right triangle, maybe I can use coordinate geometry. Let me place point A at the origin (0,0), point B at (x, 0), and point C at (0, x). Then, BC would be the hypotenuse

In [26]:
len(tokenizer.encode(line['response']))

5595